# A python solution to calculate Pv for pure substances using PR-EOS

***
**Note: Click on the `Cell` in the task bar above, then click on `Run All` to run the entire code**


In [1]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
from IPython.display import display, HTML, Markdown
import math
import copy
from functools import reduce

## The cells below solves question 1
Calculate the vapor pressure of propane together with the densities of the liquid and gas at 104°F.
Compare your answers with values from Figures 2-7, 2-12, and 3-4 (McCain Jr., W.D. The
Properties of Petroleum Fluids (2nd Edition). PennWell Publishing Company, Tulsa, OK, 1990.). 

### Utility functions

In [2]:
# utility methods
def convert_f_to_r(temp):
    return temp + 459.67

def calculate_tr(temp, temp_c):
    converted_temp = convert_f_to_r(temp)
    converted_temp_c = convert_f_to_r(temp_c)
    return converted_temp / converted_temp_c

def calculate_b(temp_c, pressure_c):
    converted_temp_c = convert_f_to_r(temp_c)
    return (0.07780 * ((10.732 * converted_temp_c) / (pressure_c)))

def calculate_ac(temp_c, pressure_c):
    converted_temp_c = convert_f_to_r(temp_c)
    return (0.45724 * ((10.732 ** 2)* (converted_temp_c ** 2)) / pressure_c)

def calculate_alpha(w, temp, temp_c):
    tr = calculate_tr(temp, temp_c)
    alpha_half = 1 + (0.37464 + (1.54226 * w) - (0.26992 * (w ** 2))) * (1 - (tr ** 0.5))
    return alpha_half ** 2

def calculate_at(ac, alpha):
    return ac * alpha

# Trial section functions
def calculate_A(aT, trial_pressure, sys_temp):
    temp = convert_f_to_r(sys_temp)
    return (aT * trial_pressure) / ((10.732 ** 2) * (temp ** 2))

def calculate_B(b, trial_pressure, sys_temp):
    temp = convert_f_to_r(sys_temp)
    return (b * trial_pressure) / (10.732 * temp)

def cubic_root_calculation_for_PR_EOS(A, B, print_original=False):
    # define coefficents
    a = 1
    b = -(1 - B)
    c = (A - (2*B) - (3*(B ** 2)))
    d = -((A*B) - (B**2) - (B**3))
    
    coefs = [a, b, c, d]
    roots = np.roots(coefs)
    # check for complex numbera and get the real part
    new_roots = []
    if not print_original:
        for number in roots:
            new_roots.append(number.real)
        return new_roots
    else:
        return list(roots)

def filter_out_complex_numbers(numbers_array):
    filtered_numbers = []
    for num in numbers_array:
        if num.imag == 0:
            filtered_numbers.append(num.real)
    return filtered_numbers
    
def get_highest_or_lowest_root(roots, phase):
    try:
        if type(roots) != list or len(roots) == 0:
            raise Exception('The roots have to be a list and not empty')
        else:
            if len(roots) > 1:
                lowest = min(roots)
                highest = max(roots)
                return highest if phase == 'gas' else lowest
            else:
                return roots[0]
    except Exception as error:
        text = f"<div style='color:red'>{error}</div>"
        display(HTML(text))
            

def solve_for_fg_fl(z, B, A, trial_pressure):
    z = z
    B = B
    A = A
    p = trial_pressure
    first_part = math.log(z - B)
    second_part = (A / ((2**1.5)*B))
    numerator = (z + (((2**0.5) + 1) * B))
    denominator = (z - (((2**0.5) - 1) * B))
    log_part = math.log(numerator / denominator)
    f_value = math.exp(z - 1 - first_part - (second_part * log_part)) * trial_pressure
    
    return f_value

def render_table(data, title, transpose=False, change_indexing_id=False, new_index="S/N", should_display=True):
    df = pd.DataFrame(data)
    if change_indexing_id:
        df.set_index(new_index, inplace=True)
    if should_display:
        display(df)
    return df

def render_multi_index_2D_table(data, outer_column_1, outer_column_2, inner_1, inner_2, unique, should_display=True):
    df = pd.DataFrame(data)

    # create a MultiIndex for liquid and gas columns
    columns = pd.MultiIndex.from_product([[outer_column_1, outer_column_2], [inner_1, inner_2]], names=['', ''])

    # create a new DataFrame with the desired columns and index
    new_df = pd.DataFrame(columns=columns, index=df.index)

    # fill in the values for the new DataFrame
    for i, row in df.iterrows():
        new_df.loc[i, (outer_column_1, inner_1)] = row[outer_column_1][inner_1]
        new_df.loc[i, (outer_column_1, inner_2)] = row[outer_column_1][inner_2]
        new_df.loc[i, (outer_column_2, inner_1)] = row[outer_column_2][inner_1]
        new_df.loc[i, (outer_column_2, inner_2)] = row[outer_column_2][inner_2]

    # add the component column back to the new DataFrame
    new_df.insert(0, unique, df[unique])

    # display the new DataFrame
    if should_display:
        display(new_df)
    return new_df

### Working data for question one
This will contain the list of species to be considered, their critical properties and acentric factor

In [3]:
# working data question 1
species_list = {
    "1": { "name": "iso-butane", "pc": 527.9, "tc": 274.46, "w": 0.1852, "Mw": 58.123 },
    "2": { "name": "propane", "pc": 616.0, "tc": 206.06, "w": 0.1522, "Mw": 49.097 }
}

### Widgets and interaction

In [4]:
# Create some widgets
title = widgets.HTML(
    value="<h2>Select the species and input the temperature</h2>"
)
temperature = widgets.FloatText(
    value="104", 
    description="Farenheit", 
    layout={'width': '25%', 'margin': '10px 0px 10px 10px'},
)
pressure_widget = widgets.FloatText(
    value="100", 
    min=14.7, 
    description="psia", 
    layout={'width': '25%', 'margin': '10px 0px 10px 10px'},
)

# loop through to key for the dropdown options
dropdown_options = [(value["name"], int(key)) for key, value in species_list.items()]

# specie dropdown
specie_dropdown = widgets.Dropdown(
    options=dropdown_options,
    value=2,
    description='Specie', 
    layout={'width': '25%', 'margin': '10px 0px 10px 10px'},
)

# interact with widgets
def on_value_change(sys_temp, specie, initial_trial_pressure):
    selected_specie = species_list[f"{specie}"]
    pc = selected_specie['pc']
    tc = selected_specie['tc']
    tr = round(calculate_tr(sys_temp, tc), 5)
    b = round(calculate_b(tc, pc), 4)
    ac = round(calculate_ac(tc, pc), 0)
    w = selected_specie['w']
    alpha = round(calculate_alpha(w, sys_temp, tc), 4)
    aT = round(calculate_at(ac, alpha), 0)
    
#     create table for the initially calculated values
    initial_calulated_values = {
        "Tr": [tr],
        "b": [b],
        "ac": [ac],
        "alpha": [alpha],
        "at": [aT]
    }
    
    render_table(initial_calulated_values, 'PR-EOS REQUIRED COEFFICIENTS')
    
#     iteration code
    iteration_table_data = {
#         "z1": [],
#         "z2": [],
#         "z3": [],
        "zg": [],
        "zl": [],
        "fg": [],
        "fl": [],
        "fg - fl": [],
        "pressure": [],
        "Vmg": [],
        "Vml": [],
        "ρg": [],
        "ρl": [],
    }
    
    fg = 0
    fl = 1000
    pressure = initial_trial_pressure
    tolerance = 0.01
    while abs(fg - fl) >= tolerance:
        A = round(calculate_A(aT, pressure, sys_temp), 5)
        B = round(calculate_B(b, pressure, sys_temp), 6)
        roots = cubic_root_calculation_for_PR_EOS(A, B)

#         iteration_table_data["z1"].append(roots[0])
#         iteration_table_data["z2"].append(roots[1])
#         iteration_table_data["z3"].append(roots[2])

        z_gas = get_highest_or_lowest_root(roots, 'gas')
        z_liquid = get_highest_or_lowest_root(roots, 'liquid')
        iteration_table_data["zg"].append(round(z_gas, 4))
        iteration_table_data["zl"].append(round(z_liquid, 4))

        fg = solve_for_fg_fl(z_gas, B, A, pressure)
        fl = solve_for_fg_fl(z_liquid, B, A, pressure)
        iteration_table_data["fg"].append(round(fg, 4))
        iteration_table_data["fl"].append(round(fl, 4))
        
        iteration_table_data["pressure"].append(round(pressure, 4))
        iteration_table_data["fg - fl"].append(round(abs(fg - fl), 4))
        
        Vmg = (z_gas * 10.732 * convert_f_to_r(sys_temp)) / pressure
        Vml = (z_liquid * 10.732 * convert_f_to_r(sys_temp)) / pressure
        iteration_table_data["Vmg"].append(round(Vmg, 4))
        iteration_table_data["Vml"].append(round(Vml, 4))
        
        den_g = ( (pressure * selected_specie["Mw"]) / (z_gas * 10.73 * convert_f_to_r(sys_temp))) / 1000
        iteration_table_data["ρg"] = den_g
        
        den_l = ( (pressure * selected_specie["Mw"]) / (z_liquid * 10.73 * convert_f_to_r(sys_temp))) / 1000
        iteration_table_data["ρl"] = den_l
        
        pressure = pressure + 0.01
        
    iteration_table = render_table(iteration_table_data, "ITERATION RESULTS")
    calculated_data = iteration_table.tail(1)
    chart_data = { "z": [0.05], "pressure": [200], "ρ": [0.03] }
    
    display(iteration_table.tail(1).style.set_caption('Calculated Data'))
    display(pd.DataFrame(chart_data).style.set_caption('Reading from Charts'))
    
    
# Display widgets    
display(title)
trap_return = widgets.interact(on_value_change, sys_temp=temperature, specie=specie_dropdown, initial_trial_pressure=pressure_widget)

HTML(value='<h2>Select the species and input the temperature</h2>')

interactive(children=(FloatText(value=104.0, description='Farenheit', layout=Layout(margin='10px 0px 10px 10px…

## The cells below solves question 2

Calculate the compositions and densities of the equilibrium liquid and gas of the mixture given
below at 160°F and 2000 psia. Use binary interaction coefficients of 0.021 for methane-n-butane,
0.032 for methane-n-decane, and 0.0 for n-butane-n-decane. The pre-specified tolerance is for 0.098.
Note: The first trial must be done manually; then use your own coded program to perform the
necessary iterations. 

|Component| Composition, mole fraction|
|----|----|
|Methane| 0.5523|
|n-Butane| 0.3630|
|n-Decane| 0.0838|
||1.0000| 

### Working data for question 2

In [5]:
# working data question 2
mixtures = {
    "example": {
        "data" : [
            {"component": "C\u2081", "name": "Methane", "zj": 0.5301, "tc": -116.67, "pc": 666.4, "w": 0.0104, "k": 3.622},
            {"component": "n-C\u2084", "name": "n-Butane", "zj": 0.1055, "tc": 305.63, "pc": 550.6, "w": 0.1995, "k": 0.211},
            {"component": "n-C\u2081\u2080", "name": "n-Decane", "zj": 0.3644, "tc": 652.03, "pc": 305.2, "w": 0.4894, "k": 0.02},
        ],
        "bip_matrix": [
            [0.0, 0.02, 0.04],   # methane
            [0.02, 0.0, 0.0],    # n-butane
            [0.04, 0.0, 0.0],    # n-decane
        ]
    },
    "assignment": {
        "data": [
            {"component": "C\u2081", "name": "Methane", "zj": 0.5523, "tc": -116.67, "pc": 666.4, "w": 0.0104, "k": 2},
            {"component": "n-C\u2084", "name": "n-Butane", "zj": 0.3630, "tc": 305.63, "pc": 550.6, "w": 0.1995, "k": 0.19},
            {"component": "n-C\u2081\u2080", "name": "n-Decane", "zj": 0.0838, "tc": 652.03, "pc": 305.2, "w": 0.4894, "k": 0.018}
        ],
        "bip_matrix": [
            [0.0, 0.021, 0.032],   # methane
            [0.021, 0.0, 0.0],    # n-butane
            [0.032, 0.0, 0.0],    # n-decane
        ]
    }
}

In [11]:
experiment_results = [
    {"component": "C\u2081", "name": "Methane", "xj": 0.485, "yj": 0.826 },
    {"component": "n-C\u2084", "name": "n-Butane", "xj": 0.412, "yj": 0.167 },
    {"component": "n-C\u2081\u2080", "name": "n-Decane", "xj": 0.103, "yj": 0.0063 }
]

In [12]:
# calculate_xj of each component
def calculate_xj(z, k, ng):
    return z / (1 + (ng * (k - 1)))

def calculate_yj(z, k, nl):
    return z / (1 + (nl * ((1/k) - 1)))

def sum_of_a_key_in_list_of_dict(key, list):
    total_sum = 0
    for item in list:
        total_sum += item[key]
    return total_sum

def keys_to_list_of_dict(key, data):
    return {component["component"]: component[key] for component in data}

def update_key_value(data_list, check_value, target_key, new_value):
    for component in data_list:
        if component['component'] == check_value:
            component[target_key] = new_value
            break

def calculate_aT_per_phase(y, aT, kappa):
    """y is an list of the yj, aT is an list of aTj and kappa is a matrix (2D list) of te BIP"""
    n = len(y)
    aT_sum = 0.0
    for i in range(n):
        for j in range(n):
            aT_sum += y[i] * y[j] * ((aT[i] * aT[j])**(0.5)) * (1.0 - (kappa[i][j]))
    return aT_sum

def calcuate_b_per_phase(y, b):
    n = len(y)
    b_sum = 0.0
    for i in range(n):
        b_sum += y[i] * b[i]
    return b_sum

In [13]:
# widgets
title_2 = widgets.HTML(
    value="<h2>Select the mixture and input the temperature</h2>"
)

temperature_2 = widgets.FloatText(
    value="160", 
    description="Farenheit", 
    layout={'width': '25%', 'margin': '10px 0px 10px 10px'},
)
pressure_2 = widgets.FloatText(
    value="2000", 
    description="psia", 
    layout={'width': '25%', 'margin': '10px 0px 10px 10px'},
)

mixture_dropdown_options = [(value, index) for index, value in enumerate(mixtures.keys())]
# mixture dropdown
mixture_dropdown = widgets.Dropdown(
    options=mixture_dropdown_options,
    value=1,
    description='Mixture', 
    layout={'width': '25%', 'margin': '10px 0px 10px 10px'},
)

ng_widget = widgets.FloatText(
    value="0.2", 
    step="0.0001",
    description="ng", 
    layout={'width': '25%', 'margin': '10px 0px 10px 10px'},
)

# The currently selected mixture and its default
current_mixture = []

# define the default for the current selected mixture from the dropdown
current_mixture = mixtures[list(mixtures.keys())[mixture_dropdown.value]]['data']
    
# dynamic widgets
dyn_widget_dict = {f"{component['component']}": widgets.FloatText(value=component['k'], description=f"k ({component['component']})", layout={'width': '25%', 'margin': '10px 0px 10px 10px'}) for component in current_mixture}
dyn_widget_args = [dyn_widget_dict[key] for key in dyn_widget_dict]

### Calculate the coefficients of the components in the mixture

In [17]:
# functions
def on_mixture_temp_change(mixture, sys_temp, sys_pressure, ng, **args):
    current_mixture = mixtures[list(mixtures.keys())[mixture]]['data']
    step_2_data = []
    
    for component in current_mixture:
        pc = component['pc']
        tc = component['tc']
        w = component['w']
        tr = round(calculate_tr(sys_temp, tc), 5)
        component['tr'] = tr
        b = round(calculate_b(tc, pc), 4)
        component['b'] = b
        ac = round(calculate_ac(tc, pc), 0)
        component['ac'] = ac
        alpha = round(calculate_alpha(w, sys_temp, tc), 4)
        component['alpha'] = alpha
        aT = round(calculate_at(ac, alpha), 0)
        component['aT'] = aT
        
        short_dict = {
            "component": component['component'],
            "k": component['k'],
            "zj": component['zj']
        }
        step_2_data.append(short_dict)
    
    initial_coef_data_df = render_table(current_mixture, "PR-EOS REQUIRED COEFFICIENTS", False, True, 'component', False)
    initial_coef_data_df['tc'] = initial_coef_data_df['tc'] + 459.67
    display(initial_coef_data_df)
    step_2(step_2_data)

    
def step_2(step_2_data):
    """The outer while loop to get values of kj"""
    display_tables = [0,1,2,3,4,5,6]
    tolerance = 0.098
    err_sum = 1000000
    
    while err_sum > tolerance:
        # get the initial k values and ng
        ng = ng_widget.value
        initializing_data = copy.deepcopy(step_2_data)
        
        """Inner while loop for the ng"""
        sum_xj = 0
        while not math.isclose(sum_xj, 1, rel_tol=0.0001):
            for comp in initializing_data:
                comp['xj'] = calculate_xj(comp['zj'], comp['k'], ng)
                comp['yj'] = calculate_yj(comp['zj'], comp['k'], 1 - ng)

            # update each guess
            ng += 0.0001
            sum_xj = sum_of_a_key_in_list_of_dict('xj', initializing_data)
            sum_yj = sum_of_a_key_in_list_of_dict('yj', initializing_data)

        # outside the ng while loop     
        composition_df = render_table(initializing_data,'Calculated Xji', transpose=False,change_indexing_id=True,new_index='component',should_display=False)
        composition_df.loc['∑xj, yj'] = ['', '', sum_xj, sum_yj]
        display_tables[0] = composition_df
        display_tables[1] = pd.DataFrame([{"ng": ng}]).transpose()

        # merge initializing_data and step_2_data
        merged_list = [dict1 | dict2 for dict1 in current_mixture for dict2 in initializing_data if dict1['component'] == dict2['component']]

        # continue to step 3 to return the phase coeffiecents
        phase_data = step_3_phase_calc(merged_list)
        display_tables[2] = pd.DataFrame(phase_data).transpose()

        # step 4 gets the z factors
        z_factors_phase_data = step_4_z_factor_calc(phase_data)
        display_tables[3] = pd.DataFrame(z_factors_phase_data).transpose()

        # step 5 calculating composition dependent corf for each component
        # n/b: the below function call actually mutates merged list
        component_coef_data = step_5_composition_coef(merged_list, phase_data)
        display_tables[4] = render_multi_index_2D_table(component_coef_data, 'liquid', 'gas', 'Aj', 'Bj', 'component', False)

        # step six the fugacity corfficients for liquid and gas
        fugacity_coef_data = step_six_fugacity_coef(component_coef_data, merged_list, z_factors_phase_data)
        display_tables[5] = pd.DataFrame(fugacity_coef_data) 

        k_factor_data = step_seven_k_factors_and_errors(fugacity_coef_data, initializing_data)
        k_factor_df = pd.DataFrame(k_factor_data['data'])
        k_factor_df.loc['∑ε'] = ['', '', '', '', k_factor_data['err_sum']]
        display_tables[6] = k_factor_df
        
        # update the k_values and err_sum
        err_sum = k_factor_data['err_sum']
        for comp in initializing_data:
            for fug_comp in k_factor_data['data']:
                if comp['component'] == fug_comp['component']:
                    comp['k'] = fug_comp['Kj']
    
    for each in display_tables:
        display(each)
    display(HTML(f"<b>ε: {err_sum} is less than tolerance {tolerance}</b>"))
    display(HTML(f"<h3>Previuosly Calculated values</h3>"))
    display(display_tables[0])
    display(HTML(f"<h3>Experimental results</h3>"))
    exp_df = pd.DataFrame(experiment_results)
    exp_df.set_index('component', inplace=True)
    display(exp_df)
    
    
def step_3_phase_calc(step_3_data):
    """liquid and gas phase coeffiecient"""
    phase_data = { "liquid": {}, "gas": {} }
    
    aTjs = keys_to_list_of_dict('aT', step_3_data)
    aTjs_list = list(aTjs.values())
    bjs = keys_to_list_of_dict('b', step_3_data)
    bjs_list = list(bjs.values())
    current_bip = mixtures[list(mixtures.keys())[mixture_dropdown.value]]['bip_matrix']
    
    # for liquid phase
    xjs = keys_to_list_of_dict('xj', step_3_data)
    xjs_list = list(xjs.values())
    aT_liquid = calculate_aT_per_phase(xjs_list, aTjs_list, current_bip)
    b_liquid = calcuate_b_per_phase(xjs_list, bjs_list)
    A_liquid = calculate_A(aT_liquid, pressure_2.value, temperature_2.value)
    B_liquid = calculate_B(b_liquid, pressure_2.value, temperature_2.value )
    liquid = { 'aT': aT_liquid, 'b': b_liquid, "A": A_liquid, "B": B_liquid }
    phase_data['liquid'] = liquid
    
    # for gas phase
    yjs = keys_to_list_of_dict('yj', step_3_data)
    yjs_list = list(yjs.values())
    aT_gas = calculate_aT_per_phase(yjs_list, aTjs_list, current_bip)
    b_gas = calcuate_b_per_phase(yjs_list, bjs_list)
    A_gas = calculate_A(aT_gas, pressure_2.value, temperature_2.value)
    B_gas = calculate_B(b_gas, pressure_2.value, temperature_2.value )
    gas = { 'aT': aT_gas, 'b': b_gas, "A": A_gas, "B": B_gas }
    phase_data['gas'] = gas
    
    return phase_data

def step_4_z_factor_calc(phase_data):
    liquid_phase_roots = cubic_root_calculation_for_PR_EOS(phase_data['liquid']['A'], phase_data['liquid']['B'], True)
    liquid_real_roots = filter_out_complex_numbers(liquid_phase_roots)
    z_liquid = get_highest_or_lowest_root(liquid_real_roots, 'liquid')
    phase_data['liquid']['z'] = z_liquid
    
    gas_phase_roots = cubic_root_calculation_for_PR_EOS(phase_data['gas']['A'], phase_data['gas']['B'], True)
    gas_real_roots = filter_out_complex_numbers(gas_phase_roots)
    z_gas = get_highest_or_lowest_root(gas_real_roots, 'gas')
    phase_data['gas']['z'] = z_gas
    
    return phase_data

def step_5_composition_coef(comp_list, phase_data):
    current_bip = mixtures[list(mixtures.keys())[mixture_dropdown.value]]['bip_matrix']
    
    def calculate_summation(target_comp, phase, bip_to_use):
        """The phase can either be xj for liquid and yj for gas"""
        sum = 0
        for ind, value in enumerate(comp_list):
            sum += value[phase] * ((target_comp['ac'] * value['ac'] * target_comp['alpha'] * value['alpha']) ** 0.5) * (1 - bip_to_use[ind])
        return sum
    
    for index, comp in enumerate(comp_list):
        liquid_Bj = comp['b'] / phase_data['liquid']['b']
        liquid_Aj = (2 * calculate_summation(comp, 'xj', current_bip[index])) / phase_data['liquid']['aT']
        comp['liquid'] = {'Aj': liquid_Aj, 'Bj': liquid_Bj}
        
        gas_Bj = comp['b'] / phase_data['gas']['b']
        gas_Aj = (2 * calculate_summation(comp, 'yj', current_bip[index])) / phase_data['gas']['aT']
        comp['gas'] = {'Aj': gas_Aj, 'Bj': gas_Bj}
    
    new_list = [{'component': d['component'], 'liquid': d['liquid'], 'gas': d['gas']} for d in comp_list]
    return new_list

def step_six_fugacity_coef(component_coef_data, original_data, phase_data):
    merged = [dict1 | dict2 for dict1 in component_coef_data for dict2 in original_data if dict1['component'] == dict2['component']]
    
    def solve_fug(comp, phase):
        term_1 = comp[phase]['Bj'] * (phase_data[phase]['z'] - 1)
        term_2 = math.log(phase_data[phase]['z'] - phase_data[phase]['B'])
        term_3 = (phase_data[phase]['A'] / ((2**1.5) * phase_data[phase]['B']))
        term_4 = (comp[phase]['Aj'] - comp[phase]['Bj'])
        numerator = phase_data[phase]['z'] + (((2 ** 0.5) + 1) * phase_data[phase]['B'])
        denom = phase_data[phase]['z'] - (((2 ** 0.5) - 1) * phase_data[phase]['B'])
        term_5 = math.log(numerator / denom)
        
        result = math.exp(term_1 - term_2 - ( term_3 * term_4 * term_5 ))
        return result
    fug_data = []
    for component in merged:
        fug_l = solve_fug(component, 'liquid')
        fug_g = solve_fug(component, 'gas')
        
        fug_data.append({"component": component['component'], "fug_l": fug_l, "fug_g": fug_g})
    return fug_data
    
def step_seven_k_factors_and_errors(fugacity_coef_data, initializing_data):
    for comp in fugacity_coef_data:        
        comp['Kj'] = comp['fug_l'] / comp['fug_g']
        for initial_comp in initializing_data:
            err_sum = 0
            if comp['component'] == initial_comp['component']:
                comp['ε'] = ((initial_comp['k'] - comp['Kj']) ** 2) / (initial_comp['k'] * comp['Kj'])
                err_sum += comp['ε']
    
    return { 'data': fugacity_coef_data, 'err_sum': err_sum }
    

display(title_2)   
# interactive_widgets = widgets.interact_manual(
interactive_widgets = widgets.interactive(
    on_mixture_temp_change, 
    mixture=mixture_dropdown, 
    sys_temp=temperature_2,
    sys_pressure=pressure_2,
    ng=ng_widget,
    **dyn_widget_dict,
)
for arg in dyn_widget_args:
    arg.observe(lambda change: interactive_widgets.update(), 'value')

display(interactive_widgets)


HTML(value='<h2>Select the mixture and input the temperature</h2>')

interactive(children=(Dropdown(description='Mixture', index=1, layout=Layout(margin='10px 0px 10px 10px', widt…